In [ ]:
!export LC_ALL=en_US.UTF-8
!CUDA_VISIBLE_DEVICES=0,1,2,3
from embedding import *

In [ ]:
import json
from pprint import pprint
import re
import numpy as np

import torch
try:
    import nltk
except:
    !pip install nltk
    import nltk
    nltk.download('punkt')
    nltk.download('stopwords')

In [ ]:
data = json.load(open('../train-v1.1.json'))

In [ ]:
try:
    from nltk.corpus import stopwords
except:
    nltk.download('stopwords')
    nltk.download('punkt')
nltkStopWords = stopwords.words('english')
punctuations = [',', '?', '.', '-',]

In [ ]:
def extractor(data):
    contexts = []
    qas = []
    for i in range(len(data["data"])):
        for j in range(len(data["data"][i]["paragraphs"])):
            contexts.append(data["data"][i]["paragraphs"][j]["context"])
            qas.append(data["data"][i]["paragraphs"][j]["qas"])
    return (contexts,qas)

In [ ]:
CapPassage = False

from nltk.tokenize import word_tokenize
contexts,qas = extractor(data)

def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll
    return (-1,-1)

def capPassage(passage,answer,cap_length = 30):
    y = np.zeros(cap_length)
    left,right = find_sub_list(answer,passage)
    if(left==-1):
        return passage[0:cap_length]
    left = left - int((cap_length - len(answer))/2)
    right = right + int((cap_length + len(answer))/2)
    if(left < 0):
        left = 0
    if(right > len(passage)):
        right = len(passage)
    return passage[left:right]
    
def findAnsVec(answer,passage):
    ans = np.zeros((len(passage)))
    start,end = find_sub_list(answer,passage)
    if(start==-1):
        start = passage.index(answer[0])
        end = start + len(answer)
    ans[start:end] = 1
    return ans

In [ ]:
def prepareSetForInference(context, answer, ground_truth_answer=None, useGroundTruth=False):
    
    passage = word_tokenize(context.lower())
    doc_len = len(passage)
    
    if(useGroundTruth):
        ans_condition = word_tokenize(ground_truth_answer.lower())
    else:
        ans_condition = word_tokenize(answer.lower())
    
    ans_len = len(ans_condition)
        
    start,end = find_sub_list(ans_condition,passage)

    if start == -1:
            print("Couldn' Find answer in text, Please try again !!")
            return None
    
    
    document_token = np.zeros((1, doc_len), dtype=np.int32)
    document_length = np.zeros(1, dtype=np.int32)
    answer_label = np.zeros((1, doc_len), dtype=np.int32)
    answer_mask = np.zeros((1, ans_len, doc_len), dtype=np.int32)
    answer_length = np.zeros(1, dtype=np.int32)
    
    document_length[0] = doc_len
    answer_length[0] = ans_len
    answer_label[0,start:end+1] = 1
    
    for i in range(doc_len):
        document_token[:,i] = look_up_word(passage[i])
    for i in range(ans_len):
        answer_mask[:,i,start+i] = 1
    
    return {
        "document_token" : document_token,
        "document_length" : document_length,
        "answer_label" : answer_label,
        "answer_mask" : answer_mask,
        "answer_length" : answer_length,
    }

In [ ]:
def inference(context, answer = []):
    
    data_dict = prepareSetForInference(context, answer)
    
    use_attention = True
    use_ground_truth = True
    maxDocLenForBatch = max_document_len

    answer_encoder_hidden_inf = answerEncoder2.hiddenState
    question_encoder_hidden_inf = questionEncoder2.hiddenState
    question_decoder_hidden_inf = questionDecoder2.hiddenState
    attention_hidden_inf = attention.hidden_state
    question_decoder_hidden_inf = questionDecoder2.hiddenState
    
    
    
    
    inp = Variable(torch.from_numpy(data_dict["document_token"]).long())
    labels = Variable(torch.from_numpy(data_dict["answer_label"])).long() #Let's see if we want to use or not
    if use_cuda:
        labels = labels.cuda()

    embedded_inp = embedder(inp)

    if use_cuda:
        embedded_inp = embedded_inp.cuda()


    answer_encoder_hidden_inf = repackage_hidden(answer_encoder_hidden_inf)
    answer_tags, answer_outputs, answer_encoder_hidden_inf = answerEncoder(embedded_inp, answer_encoder_hidden_inf)
    
    
    
    if use_cuda:
        answer_outputs = answer_outputs.cuda()
        answer_tags = answer_tags.cuda()


    question_encoder_input = Variable(torch.from_numpy(data_dict["answer_mask"]))
    if use_cuda:
        question_encoder_input = question_encoder_input.cuda()   #CHECK IF THIS IS RIGHT

    question_encoder_hidden_inf = repackage_hidden(question_encoder_hidden) ##Get this from saved model

    t_answer_mask = Variable(torch.from_numpy(data_dict["answer_mask"])).float()
    if use_cuda:
        t_answer_mask = t_answer_mask.cuda()

    question_encoder_input = torch.matmul(t_answer_mask[:,0:data_dict["answer_length"],:], answer_outputs.float())
    
    _ , question_encoder_hidden_inf = questionEncoder(question_encoder_input[:,0:data_dict["answer_length"],:], question_encoder_hidden_inf)

    if type(question_decoder_hidden_inf) == Variable:
        question_decoder_hidden_inf = repackage_hidden(question_decoder_hidden_inf)
    if(type(attention_hidden) == Variable):
        attention_hidden_inf = repackage_hidden(attention_hidden_inf)

    question_loss = 0
    question_decoder_hidden_inf = question_encoder_hidden_inf

    ###embedded_inputs = embedder(torch.from_numpy(batch_input[5][batch_num][i]).long()).cuda()

    maxGenQuesLen = 20
    currGenQuestionLen = 0

    current_question_token = START_TOKEN
    questionGenerated = []
    while currGenQuestionLen < maxGenQuesLen and current_question_token != END_TOKEN:
        question_token_embedding = embedder(Variable(torch.from_numpy(np.array([current_question_token])).long()))
        if use_cuda:
            question_token_embedding = question_token_embedding.cuda()

        if use_attention:
            attn_output, Attention_Weights = attention(question_decoder_hidden_inf.squeeze(0).squeeze(0), attention_hidden_inf.squeeze(0), answer_outputs.squeeze(0))
            decoder_output, attention_hidden_inf = questionDecoder(
                question_token_embedding.unsqueeze(1), attn_output)
        else:
            decoder_output, question_decoder_hidden_inf = questionDecoder(
                question_token_embedding.unsqueeze(1),
                question_decoder_hidden_inf)

        final_output = questionGenerator(decoder_output)
        current_question_token = np.argmax(final_output.data)
        questionGenerated.append(look_up_token_reduced(current_question_token))
        currGenQuestionLen += 1
        
    return questionGenerated

In [ ]:
X_train_comp_all = []
X_train_comp_ans_all = []
X_train_ans_all = []
Y_train_ques_all = []
invalid = 0
X_train_ans_label_all = []
for i,context in enumerate(contexts):
    passage = word_tokenize(context.lower())
    
    a_lab = np.zeros(len(passage))
    for j,_ in enumerate(qas[i]):
        answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
        start,end = find_sub_list(answer,passage)
        if start == -1:
            continue
        a_lab[start:end+1] = 1
            
            
    for j,_ in enumerate(qas[i]):
        try:
            question = word_tokenize(qas[i][j]['question'].lower())
            answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
            
            if CapPassage:
                cappedPassage = capPassage(passage,answer)
            else:
                cappedPassage = passage
            
            X_train_comp_ans_all.append(findAnsVec(answer,passage))
            X_train_ans_label_all.append(a_lab)
            X_train_comp_all.append(cappedPassage)
            X_train_ans_all.append(answer)
            Y_train_ques_all.append(question)
        except Exception as e:
            invalid = invalid+1

In [ ]:
verboseBatchPrinting = True
averageBatchLossPrinting = True

num_epochs = 100
answer_encoder_hidden = answerEncoder.initHidden()
question_encoder_1_hidden = questionEncoder1.initHidden()
question_encoder_2_hidden = questionEncoder2.initHidden()
question_decoder_hidden = None

Attention_Weights = None
attn_output = None
attention_hidden = attention.initHidden()

for epoch in range(1, num_epochs+1):
    avg_loss = 0
    
    for batch_num in range(number_of_batches):
        
        answer_encoder_hidden = repackage_hidden(answer_encoder_hidden)
        question_encoder_1_hidden = repackage_hidden(question_encoder_1_hidden)
        question_encoder_2_hidden = repackage_hidden(question_encoder_2_hidden)
        if type(question_decoder_hidden) == Variable:
            question_decoder_hidden = repackage_hidden(question_decoder_hidden)
        if(type(attn_output) == Variable):
            attn_output = repackage_hidden(attn_output)
        if(type(attention_hidden) == Variable):
            attention_hidden = repackage_hidden(attention_hidden)
        if(type(Attention_Weights) == Variable):
            Attention_Weights = repackage_hidden(Attention_Weights)
        
        current_batch_size = len(batch_input[batch_num]["document_tokens"])
        if current_batch_size != batch_size:
            continue
        
        maxDocLenForBatch = max_document_len
        mask = np.zeros((current_batch_size, maxDocLenForBatch))
        for i in range(current_batch_size):
            mask[i][0:batch_input[batch_num]["document_lengths"][i]] = 1

        inp = Variable(torch.from_numpy(batch_input[batch_num]["document_tokens"]).long())

        labels = Variable(torch.from_numpy(batch_input[batch_num]["answer_labels_all"])).long()
        if use_cuda:
            labels = labels.cuda()

        optimizer.zero_grad()
        embedded_inp = embedder(inp)
        
        answer_outputs = Variable(torch.zeros(current_batch_size, max_document_len, hidden_size))
        answer_tags = Variable(torch.zeros(current_batch_size, max_document_len, 1))
        
        if use_cuda:
            answer_outputs = answer_outputs.cuda()
            answer_tags = answer_tags.cuda()
            embedded_inp = embedded_inp.cuda()

        answer_tags, answer_outputs, answer_encoder_hidden = answerEncoder(embedded_inp, answer_encoder_hidden)              

        t_document_mask = Variable(torch.from_numpy(mask)).float()
        if use_cuda:
            t_document_mask = t_document_mask.cuda()
        outputs = torch.mul(answer_tags.squeeze(-1),t_document_mask)


        answer_loss = criterion1(outputs, labels.float())
        
        
        
        
        
        answer_mask_1 = Variable(torch.from_numpy(batch_input[batch_num]["answer_labels"])).float().unsqueeze(-1)
        answer_mask_2 = Variable(torch.from_numpy(1 - batch_input[batch_num]["answer_labels"])).float().unsqueeze(-1)
        
        if use_cuda:
            answer_mask_1 = answer_mask_1.cuda()
            answer_mask_2 = answer_mask_2.cuda()
        
        question_encoder_input1 = torch.mul(answer_mask_1, answer_outputs.float())
        question_encoder_input2 = torch.mul(answer_mask_2, answer_outputs.float())
        
        question_encoder_1_outputs , question_encoder_1_hidden = questionEncoder1(question_encoder_input1, question_encoder_1_hidden)
        question_encoder_2_outputs , question_encoder_2_hidden = questionEncoder2(question_encoder_input2, question_encoder_2_hidden)
            
        
        

        question_loss = 0
        question_decoder_hidden = question_encoder_2_hidden
        embedded_inputs = embedder(torch.from_numpy(batch_input[batch_num]["question_input_tokens"]).long())
        output_labels = Variable(torch.from_numpy(batch_input[batch_num]["question_output_tokens"]).long())
        if use_cuda:
            embedded_inputs = embedded_inputs.cuda()
            output_labels = output_labels.cuda()

        for quesL in range(batch_input[batch_num]["question_input_tokens"].shape[1]):
            if use_attention:
                attn_output, Attention_Weights = attention(question_decoder_hidden.squeeze(0).squeeze(0), attention_hidden.squeeze(0), answer_outputs[i])
                decoder_output, attention_hidden = questionDecoder(
                    embedded_inputs[quesL:quesL+1].unsqueeze(1), attn_output)
            else:
                decoder_output, question_decoder_hidden = questionDecoder(
                    embedded_inputs[:,quesL:quesL+1,:],
                    question_decoder_hidden)

            final_output = questionGenerator(decoder_output.squeeze(1))
            
            question_loss += criterion2(final_output,
                                       output_labels[:, quesL:quesL+1].squeeze(1))


        net_loss = answer_loss + question_loss
        net_loss.backward()
        optimizer.step()


        avg_loss+= net_loss.data[0]
        if verboseBatchPrinting:
            print ('Batch: %d \t Epoch : %d\tNet Loss: %.4f \tAnswer Loss: %.4f \tQuestion Loss: %.4f'
                   %(batch_num, epoch, net_loss.data[0], answer_loss.data[0], question_loss.data[0]))



    if averageBatchLossPrinting:
        print('Average Loss after Epoch %d : %.4f'
                   %(epoch, avg_loss/number_of_batches))

In [ ]:
from collections import Counter
import operator
def findKMostFrequentWords(k):
    ctr = Counter([item for sublist in X_train_comp_all for item in sublist] + [item for sublist in Y_train_ques_all for item in sublist])
    sorted_ctr = sorted(ctr.items(), key=operator.itemgetter(1), reverse=True)
    return [item[0] for item in sorted_ctr[0:k]]

In [ ]:
wordToTake = 2000
words = findKMostFrequentWords(100000)

In [ ]:
_word_to_idx_reduced = {}
_idx_to_word_reduced = []


def _add_word_reduced(word):
    idx = len(_idx_to_word_reduced)
    _word_to_idx_reduced[word] = idx
    _idx_to_word_reduced.append(word)
    return idx


UNKNOWN_TOKEN = _add_word_reduced(UNKNOWN_WORD)
START_TOKEN = _add_word_reduced(START_WORD)
END_TOKEN = _add_word_reduced(END_WORD)




dimensions = glove.shape[1]
reduced_glove = []
reduced_glove.append(np.zeros(dimensions))
reduced_glove.append(-np.ones(dimensions))
reduced_glove.append(np.ones(dimensions))

for word in words:
    l = look_up_word(word)
    if(l != UNKNOWN_TOKEN):
        idx = _add_word_reduced(word)
        reduced_glove.append(glove[l])
        if(len(reduced_glove) == wordToTake):
            break
        
def look_up_word_reduced(word):
    return _word_to_idx_reduced.get(word, UNKNOWN_TOKEN)


def look_up_token_reduced(token):
    return _idx_to_word_reduced[token]

reduced_glove = np.array(reduced_glove)
reduced_glove.shape

In [ ]:
!hostname
print(invalid)

In [ ]:
c = list(zip(X_train_comp_all,X_train_comp_ans_all, X_train_ans_all, X_train_ans_label_all,Y_train_ques_all))
np.random.shuffle(c)
X_train_comp_all_shuffled,X_train_comp_ans_all_shuffled, X_train_ans_shuffled, X_train_ans_label_shuffled,Y_train_ques_all_shuffled = zip(*c)

In [ ]:
examples_to_take_train = 32

X_train_comp = X_train_comp_all_shuffled[0:examples_to_take_train]
X_train_comp_ans = X_train_comp_ans_all_shuffled[0:examples_to_take_train]
X_train_ans = X_train_ans_shuffled[0:examples_to_take_train]
X_train_ans_label = X_train_ans_label_shuffled[0:examples_to_take_train]
Y_train_ques = Y_train_ques_all_shuffled[0:examples_to_take_train]
answer_indices = [np.where(x==1)[0].tolist() for x in X_train_comp_ans]

In [ ]:
max_document_len = len(max(X_train_comp,key=len))
max_answer_len = len(max(X_train_ans,key=len))
max_question_len = len(max(Y_train_ques,key=len)) + 1

In [ ]:
document_tokens = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
answer_masks = np.zeros((examples_to_take_train, max_answer_len, max_document_len), dtype=np.int32)
answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
question_input_tokens = np.full((examples_to_take_train, max_question_len), END_TOKEN, dtype=np.int32)
question_output_tokens = np.full((examples_to_take_train, max_question_len), END_TOKEN, dtype=np.int32)
question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
suppression_answer = np.zeros((examples_to_take_train, reduced_glove.shape[0], 1),dtype=np.int32)
expression_contexts = np.zeros((examples_to_take_train, max_question_len,reduced_glove.shape[0]),dtype=np.int32)
expression_probabilities = np.zeros((examples_to_take_train, max_question_len,reduced_glove.shape[0]),dtype=np.float32)

In [ ]:
answer_labels[0]

In [ ]:
for i in range(examples_to_take_train):
    answer_labels[i,0:len(X_train_ans_label[i])] = X_train_ans_label[i]
    for j, word in enumerate(X_train_comp[i]):
        document_tokens[i, j] = look_up_word_reduced(word)
    document_lengths[i] = len(X_train_comp[i])

    for j, index in enumerate(answer_indices[i]):
        answer_masks[i, j, index] = 1
    answer_lengths[i] = len(answer_indices[i])
    
    #print(Y_train_ques[i])
    question_input_words = ([START_WORD] + Y_train_ques[i])
    question_output_words = (Y_train_ques[i] + [END_WORD])

    for j, word in enumerate(question_input_words):
            question_input_tokens[i, j] = look_up_word_reduced(word)
    for j, word in enumerate(question_output_words):
        question_output_tokens[i, j] = look_up_word_reduced(word)
    question_lengths[i] = len(question_input_words)
    
    for j, word in enumerate(X_train_ans[i]):
        if(word not in Y_train_ques[i]):
            suppression_answer[i, look_up_word_reduced(word),:] = 1
            
    words_to_consider_expression = set(X_train_comp[i] + nltkStopWords + punctuations)

    for j,word in enumerate(words_to_consider_expression):
        expression_contexts[i,:,look_up_word_reduced(word)] = 1
        
    for j,word in enumerate(words_to_consider_expression):
        expression_probabilities[i,:,look_up_word_reduced(word)] = len(np.where(expression_contexts[i][0] == 1)[0]) / float(wordToTake)
    expression_probabilities[i,:,np.where(expression_probabilities[i][0] == 0)[0]] = len(np.where(expression_contexts[i][0] == 0)[0]) / float(wordToTake)
    
        

In [ ]:
def create_vocabulary(data):
    flat_list = [item for sublist in data for item in sublist]
    vocabulary = sorted(set(flat_list))
    vocabulary.append("<UNK>")
    vocabulary.append("unk")
    vocabulary.append("eos")
    vocabulary = ["<EOS>"] + vocabulary
    word_to_index = { word:i for i,word in enumerate(vocabulary) }
    index_to_word = { i:word for i,word in enumerate(vocabulary) }
    return (vocabulary,word_to_index,index_to_word)

In [ ]:
print(reduced_glove.shape)
vocabulary_comp,word_to_index_comp,index_to_word_comp = create_vocabulary(X_train_comp + Y_train_ques)
print(len(vocabulary_comp))
print(word_to_index_comp["?"])
print(word_to_index_comp["what"])

In [ ]:
def create_one_hot_vector(data,vocabulary,word_to_index,index_to_word, maxLen):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,word in enumerate(data):
        if i >= maxLen:
            break
        if(word not in word_to_index):
            word = "<UNK>"
        one_hot[i][word_to_index[word]] = 1
    return one_hot

def create_one_hot_vector_from_indices(data,maxLen,vocabulary):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,indice in enumerate(data):
        if i >= maxLen:
            break
        one_hot[i][int(indice)] = 1
    return one_hot


def create_one_hot_training_Set(data,maxLen,vocabulary):
    one_hot_data = np.zeros([data.shape[0],maxLen,len(vocabulary)])
    for i in range(data.shape[0]):
        one_hot_data[i] = create_one_hot_vector_from_indices(data[i],maxLen,vocabulary)
    return one_hot_data




In [ ]:
def context_to_indices_glove(X,max_len):
    
    m = len(X)                                 
    
    X_indices = np.full([m,max_len],look_up_word_reduced(END_WORD))
    
    for i in range(m):
        j = 0
        for w in X[i]:
            if(j>=max_len):
                break;
            
            X_indices[i, j] = look_up_word_reduced(w)
            j = j+1
    return X_indices

In [ ]:
document_tokens = context_to_indices_glove(X_train_comp, max_document_len)

In [ ]:
max_document_len

## Batch Data Preparation

In [ ]:
import math
def create_batch(inputs,batch_size,shuffle=False):
    num_batches = len(inputs[0]) // batch_size + 1
    outputs = []
    for index,inp in enumerate(inputs):
    
        output = {'document_tokens':[],
                    'document_lengths':[],
                    'answer_labels':[],
                    'answer_masks': [],
                    'answer_lengths': [],
                    'question_input_tokens':[],
                    'question_output_tokens':[],
                    'question_lengths':[],
                    'suppression_answer':[],
                    'expression_contexts': [],
                    'expression_probabilities':[]}
    
        start = 0
        for i in range(num_batches):
            if i == num_batches - 1:
                end = None
            else:
                end = start+batch_size
            #maxD = max(inputs[1][start:end])
            maxD = max_document_len
            maxA = max(inputs[4][start:end])
            maxQ = max(inputs[7][start:end])
            if index == 0:
                outputs['document_tokens'] = inp[start:end,:maxD]
            elif index==1:
                outputs['document_lengths'] = inp[start:end]
            elif index == 2:
                outputs['answer_labels']=inp[start:end,:maxD]
            elif index==3:
                outputs['answer_masks']=inp[start:end,:maxA,:maxD]
            elif index==4:
                outputs['answer_lengths']=inp[start:end]
            elif index==5:
                output['question_input_tokens']=inp[start:end,:maxQ]
            elif index==6:
                output['question_output_tokens']=inp[start:end,:maxQ]
            elif index==7:
                output['question_lengths'] = inp[start:end]
            elif index==8:
                output['suppression_answer'] = inp[start:end]
            elif index==9:
                output['expression_contexts'] = inp[start:end,0:maxQ,:]
            elif index==10: 
                output['expression_probabilities'] = inp[start:end,0:maxQ,:]
            start = start + batch_size
        outputs.append(output)
    
    return outputs

def createBatch(inputs,batch_size,shuffle=False):
    outputs = []
    num_batches = math.ceil(len(inputs[0])/batch_size)
    
    for index,inp in enumerate(inputs):
        start = 0
        output = []
        for i in range(num_batches-1):
            #maxD = max(inputs[1][start:start+batch_size])
            maxD = max_document_len
            maxA = max(inputs[4][start:start+batch_size])
            maxQ = max(inputs[7][start:start+batch_size])
            if index == 0 or index == 2:
                output.append(inp[start:start+batch_size,0:maxD]) 
            elif index==3:
                output.append(inp[start:start+batch_size,0:maxA,0:maxD]) 
            elif index==5 or index==6:
                output.append(inp[start:start+batch_size,0:maxQ])
            elif index==9 or index==10:
                output.append(inp[start:start+batch_size,0:maxQ,:])
            else: 
                output.append(inp[start:start+batch_size])
            start = start + batch_size
        
        # Remaining training sample i.e. training mod batch_size
        #maxD = max(inputs[1][start:])
        maxD = max_document_len
        maxA = max(inputs[4][start:])
        maxQ = max(inputs[7][start:])
        if index == 0 or index == 2:
            output.append(inp[start:,0:maxD]) 
        elif index==3:
            output.append(inp[start:,0:maxA,0:maxD]) 
        elif index==5 or index==6:
            output.append(inp[start:,0:maxQ])
        elif index==9 or index==10:
            output.append(inp[start:,0:maxQ,:]) 
        else: 
            output.append(inp[start:])
        outputs.append(output)
    
    return outputs


In [ ]:
batch_size = 32
batch_input = createBatch([document_tokens,document_lengths,answer_labels,answer_masks,answer_lengths,question_input_tokens,question_output_tokens,question_lengths,suppression_answer,expression_contexts,expression_probabilities]
                    ,batch_size)
number_of_batches = len(batch_input[0])
print("Number of batches = ", number_of_batches)

## Model

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

In [ ]:
use_cuda = torch.cuda.is_available()
use_attention = True

In [ ]:
class Embedder(nn.Module):
    def __init__(self, input_size, output_size):
        super(Embedder, self).__init__()
        self.embedding = nn.Embedding(input_size, output_size)
        
        # TODO: Verify
        self.embedding.weight = nn.Parameter(torch.from_numpy(reduced_glove).float())
        self.embedding.weight.requires_grad = False
    def forward(self, x):
        return self.embedding(x)

In [ ]:
class AnswerEncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(AnswerEncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        
        self.gru = nn.GRU(input_size, hidden_size, batch_first= True, bidirectional=True) #Input_size = Hidden_Size
        if use_cuda:
            self.gru = self.gru.cuda()
        self.fc = nn.Linear(hidden_size*2, 1)
        if use_cuda:
            self.fc = self.fc.cuda()

    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        final_output = self.fc(output)
        final_output = F.sigmoid(final_output)
        self.hiddenState = hidden
        return final_output, output, hidden
    
    def initHidden(self):
        result = Variable(torch.zeros(2, 1, self.hidden_size)) #2 for BiDirectional
        if use_cuda:
            result = result.cuda()
        return result

In [ ]:
class QuestionEncoderRNN(nn.Module):
    
    def __init__(self,input_size, hidden_size):
        super(QuestionEncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first= True)
        if use_cuda:
            self.gru = self.gru.cuda()
    
    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        self.hiddenState = hidden
        return output, hidden
    
    def initHidden(self):
        #result = Variable(torch.zeros(1, batch_size, self.hidden_size))
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            result = result.cuda()
        return result

In [ ]:
class QuestionDecoderRNN(nn.Module):
    
    def __init__(self,input_size, hidden_size):
        super(QuestionDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first= True)
        if use_cuda:
            self.gru = self.gru.cuda()
        
    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        self.hiddenState = hidden
        return output, hidden
    
    def initHidden(self):
        #result = Variable(torch(1, batch_size, self.hidden_size))
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            result = result.cuda()
        return result
    
'''
class FCLayer(nn.Module):
    def __init__(self,input_size, output_size):
        super(FCLayer, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        
    def forward(self, x):
        return self.fc(x)
'''

class QuestionGenerationFC(nn.Module):
    def __init__(self,input_size, output_size):
        super(QuestionGenerationFC, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        if use_cuda:
            self.fc = self.fc.cuda()

    def forward(self, x):
        output = self.fc(x)
        output = F.log_softmax(output, dim=-1)
        return output

In [ ]:
class AttnDecoderRNN(nn.Module):
    
    def __init__(self, hidden_size, max_document_len, dropout_p=0.1):
        
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.dropout_p = dropout_p
        self.max_document_len = max_document_len

        self.attn_reduce = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.attn_combine = nn.Linear(self.max_document_len + self.hidden_size, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        
        if use_cuda:
            self.attn_reduce = self.attn_reduce.cuda()
            self.attn_combine = self.attn_combine.cuda()
            self.dropout = self.dropout.cuda()
        

    def forward(self, input, hidden, encoder_outputs):

        hidden = hidden.squeeze(0)
        hidden = self.dropout(hidden)
        concat = torch.cat((input.unsqueeze(0), hidden.unsqueeze(0)), 1)
        concat_reduced = self.attn_reduce(concat)
        attn_weights = F.softmax(concat_reduced, dim=1)
        attn_applied = torch.mm(encoder_outputs, attn_weights.squeeze(0).unsqueeze(1))
        output = torch.cat((hidden.unsqueeze(0), attn_applied.squeeze(1).unsqueeze(0)), 1)
        output = self.attn_combine(output).unsqueeze(0)
        output = F.relu(output)
        self.hidden_state = hidden
        return output, attn_weights

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            result = result.cuda()
        return result

In [ ]:
hidden_size = reduced_glove.shape[1]

embedder = Embedder(input_size = reduced_glove.shape[0], output_size = reduced_glove.shape[1])
#fcLayer = FCLayer(hidden_size, hidden_size)
answerEncoder = AnswerEncoderRNN(input_size = hidden_size, hidden_size=int(hidden_size/2))
questionEncoder = QuestionEncoderRNN(input_size=hidden_size, hidden_size=hidden_size)
questionDecoder = QuestionDecoderRNN(input_size=hidden_size, hidden_size=hidden_size)
questionGenerator = QuestionGenerationFC(input_size = hidden_size, output_size=reduced_glove.shape[0])
attention = AttnDecoderRNN(hidden_size= hidden_size,max_document_len = max_document_len)

answerEncoder.train()
questionEncoder.train()
questionDecoder.train()
questionGenerator.train()
attention.train()

train_param = []

for model in [answerEncoder, questionEncoder, questionDecoder, questionGenerator, attention]:
    train_param += [p for p in model.parameters() if p.requires_grad]

print("Number of trainable parameters = ", len(train_param))

optimizer = torch.optim.Adam(train_param, 0.01)
criterion1 = nn.BCELoss()
#criterion2 = nn.CrossEntropyLoss()
criterion2 = nn.NLLLoss()
#criterion2 = nn.MultiLabelSoftMarginLoss()

In [ ]:
def repackage_hidden(h):
    """Wraps hidden states in new Variables, to detach them from their history."""
    if type(h) == Variable:
        return Variable(h.data)
    else:
        return tuple(repackage_hidden(v) for v in h)

In [ ]:
verboseBatchPrinting = True
averageBatchLossPrinting = True

num_epochs = 100
answer_encoder_hidden = answerEncoder.initHidden()
question_encoder_hidden = questionEncoder.initHidden()
question_decoder_hidden = None

if use_attention:
    Attention_Weights = None
    attn_output = None
    attention_hidden = attention.initHidden()

for epoch in range(1, num_epochs+1):
    avg_loss = 0
    for batch_num in range(len(batch_input[0])):

        current_batch_size = len(batch_input[0][batch_num])
        if current_batch_size != batch_size:
            continue


        #maxDocLenForBatch = int(max(batch_input[1][batch_num]))
        maxDocLenForBatch = max_document_len
        mask = np.zeros((current_batch_size, maxDocLenForBatch))
        for i in range(current_batch_size):
            mask[i][0:batch_input[1][0][i]] = 1

        inp = Variable(torch.from_numpy(batch_input[0][batch_num]).long())

        labels = Variable(torch.from_numpy(batch_input[2][batch_num])).long()
        if use_cuda:
            labels = labels.cuda()

        optimizer.zero_grad()
        embedded_inp = embedder(inp).cuda()
        answer_encoder_hidden = repackage_hidden(answer_encoder_hidden)
        
        
        answer_outputs = Variable(torch.zeros(current_batch_size, max_document_len, hidden_size))
        answer_tags = Variable(torch.zeros(current_batch_size, max_document_len, 1))
        
        if use_cuda:
            answer_outputs = answer_outputs.cuda()
            answer_tags = answer_tags.cuda()

        for i in range(current_batch_size):
            answer_tags[i], answer_outputs[i], answer_encoder_hidden = answerEncoder(embedded_inp[i:i+1], answer_encoder_hidden)   
            

        t_document_mask = Variable(torch.from_numpy(mask)).float()
        if use_cuda:
            t_document_mask = t_document_mask.cuda()
        outputs = torch.mul(answer_tags.squeeze(-1),t_document_mask)


        answer_loss = criterion1(outputs, labels.float())


        t_answer_mask = Variable(torch.from_numpy(batch_input[3][batch_num])).float()
        if use_cuda:
            t_answer_mask = t_answer_mask.cuda()

        question_encoder_input = torch.matmul(t_answer_mask, answer_outputs.float())
        question_encoder_hidden_batch = Variable(torch.zeros(1,current_batch_size,questionEncoder.hidden_size))
        if use_cuda:
            question_encoder_hidden_batch = question_encoder_hidden_batch.cuda()

        question_encoder_hidden = repackage_hidden(question_encoder_hidden)
        for i in range(current_batch_size):
            _ , question_encoder_hidden = questionEncoder(question_encoder_input[i:i+1,0:batch_input[4][batch_num][i],:], question_encoder_hidden)
            question_encoder_hidden_batch[:,i:i+1,:] = question_encoder_hidden

        if type(question_decoder_hidden) == Variable:
            question_decoder_hidden = repackage_hidden(question_decoder_hidden)
        if(type(attn_output) == Variable):
            attn_output = repackage_hidden(attn_output)
        if(type(attention_hidden) == Variable):
            attention_hidden = repackage_hidden(attention_hidden)
        if(type(Attention_Weights) == Variable):
            Attention_Weights = repackage_hidden(Attention_Weights)
        
        question_loss = 0
        for i in range(current_batch_size):
            question_decoder_hidden = question_encoder_hidden_batch[:,i:i+1,:].clone()
            embedded_inputs = embedder(torch.from_numpy(batch_input[5][batch_num][i]).long())
            output_labels = Variable(torch.from_numpy(batch_input[6][batch_num][i]).long())
            if use_cuda:
                embedded_inputs = embedded_inputs.cuda()
                output_labels = output_labels.cuda()

            for quesL in range(batch_input[7][batch_num][i]):
                if use_attention:
                    attn_output, Attention_Weights = attention(question_decoder_hidden.squeeze(0).squeeze(0), attention_hidden.squeeze(0), answer_outputs[i])
                    decoder_output, attention_hidden = questionDecoder(
                        embedded_inputs[quesL:quesL+1].unsqueeze(1), attn_output)
                else:
                    decoder_output, question_decoder_hidden = questionDecoder(
                        embedded_inputs[quesL:quesL+1].unsqueeze(1),
                        question_decoder_hidden)

                final_output = questionGenerator(decoder_output)
                output_label = Variable(torch.zeros(1,reduced_glove.shape[0]))
                if use_cuda:
                    output_label = output_label.cuda()
                output_label[:,batch_input[5][batch_num][i][quesL]] = 1
                question_loss += criterion2(final_output.squeeze(0),
                                           output_labels[quesL:quesL+1])
                ##question_loss += criterion2(final_output.squeeze(0), output_label)


        net_loss = answer_loss + question_loss
        net_loss.backward(retain_graph=True)
        optimizer.step()


        avg_loss+= net_loss.data[0]
        if verboseBatchPrinting:
            print ('Batch: %d \t Epoch : %d\tNet Loss: %.4f \tAnswer Loss: %.4f \tQuestion Loss: %.4f'
                   %(batch_num, epoch, net_loss.data[0], answer_loss.data[0], question_loss.data[0]))



    if averageBatchLossPrinting:
        print('Average Loss after Epoch %d : %.4f'
                   %(epoch, avg_loss/number_of_batches))

In [ ]:
checkpoint = torch.load("/data/ra2630/qgen_base_40k.pt")

In [ ]:
answerEncoder2 = AnswerEncoderRNN(input_size = hidden_size, hidden_size=int(hidden_size/2))
questionEncoder2 = QuestionEncoderRNN(input_size=hidden_size, hidden_size=hidden_size)
questionDecoder2 = QuestionDecoderRNN(input_size=hidden_size, hidden_size=hidden_size)
questionGenerator2 = QuestionGenerationFC(input_size = hidden_size, output_size=reduced_glove.shape[0])
optimizer2 = torch.optim.Adam(train_param, 0.01)

#answerEncoder2.load_state_dict(checkpoint["answerEncoder"])
#questionEncoder2.load_state_dict(checkpoint["questionEncoder"])
#questionDecoder2.load_state_dict(checkpoint["questionDecoder"])
#questionGenerator2.load_state_dict(checkpoint["questionGenerator"])
#optimizer2.load_state_dict(checkpoint["optimizer"])

answerEncoder2 = answerEncoder
questionEncoder2 = questionEncoder
questionDecoder2 = questionDecoder
questionGenerator2 = questionGenerator
optimizer2 = optimizer

In [ ]:
def printQuestion(batch_num, example_num):
    for i in batch_input[6][batch_num][example_num]:
        print(look_up_token_reduced(i), end = ' ', sep = ' ')
    print("")
        
def printAnswer(batch_num, example_num):
    for i in batch_input[3][batch_num][example_num][0 : batch_input[4][batch_num][example_num]]:
        for j in (np.where(i==1)[0]):
            print(look_up_token_reduced(batch_input[0][batch_num][example_num][j]))
    print("")
        
def printContext(batch_num, example_num):
    for i in batch_input[0][batch_num][example_num]:
        print(look_up_token_reduced(i), end = ' ', sep = ' ')
    print("")

In [ ]:
batch_num = 0
example_num=0
print("Context : ")
printContext(batch_num, example_num)
print("Question : ")
printQuestion(batch_num, example_num)
print("Answer : ")
printAnswer(batch_num, example_num)

In [ ]:
question_token = START_TOKEN
question_token_embedding = embedder(Variable(torch.from_numpy(np.array([question_token])).long())).cuda()
qLen = 0
while qLen <= 20:
    decoder_output, question_decoder_hidden = questionDecoder(
        question_token_embedding.unsqueeze(1),
        question_decoder_hidden)
    final_output = questionGenerator(decoder_output)
    question_token = np.argmax(final_output.data)
    print(question_token)
    question_token_embedding = embedder(Variable(torch.from_numpy(np.array([question_token])).long())).cuda()
    qLen=qLen + 1
    print(look_up_token_reduced(question_token))

In [ ]:
#document_tokens = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
#document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
#answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
#answer_masks = np.zeros((examples_to_take_train, max_answer_len, max_document_len), dtype=np.int32)
#answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
#question_input_tokens = np.full((examples_to_take_train, max_question_len), END_TOKEN, dtype=np.int32)
#question_output_tokens = np.full((examples_to_take_train, max_question_len), END_TOKEN, dtype=np.int32)
#question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
#suppression_answer = np.zeros((examples_to_take_train, reduced_glove.shape[0], 1),dtype=np.int32)
#expression_contexts = np.zeros((examples_to_take_train, max_question_len,reduced_glove.shape[0]),dtype=np.int32)
#expression_probabilities = np.zeros((examples_to_take_train, max_question_len,reduced_glove.shape[0]),dtype=np.float32)

In [ ]:
answerEncoder2 = answerEncoder
questionEncoder2 = questionEncoder
questionDecoder2 = questionDecoder
questionGenerator2 = questionGenerator
attention2 = attention
optimizer2 = optimizer

In [ ]:
for i in range(32):
    printQuestion(0,i)
    print(inference(0,i,""))

In [ ]:
def inference(batch_number, example_number, context, answer = []):
    
    use_attention = True
    use_ground_truth = True
    maxDocLenForBatch = max_document_len

    answer_encoder_hidden_inf = answerEncoder2.hiddenState
    question_encoder_hidden_inf = questionEncoder2.hiddenState
    question_decoder_hidden_inf = questionDecoder2.hiddenState
    attention_hidden_inf = attention.hidden_state
    question_decoder_hidden_inf = questionDecoder2.hiddenState
    
    
    
    
    inp = Variable(torch.from_numpy(batch_input[0][batch_number][example_number:example_number+1]).long())
    labels = Variable(torch.from_numpy(batch_input[2][batch_number][example_number:example_number+1])).long() #Let's see if we want to use or not
    if use_cuda:
        labels = labels.cuda()

    embedded_inp = embedder(inp)

    if use_cuda:
        embedded_inp = embedded_inp.cuda()


    answer_encoder_hidden_inf = repackage_hidden(answer_encoder_hidden_inf)
    answer_tags, answer_outputs, answer_encoder_hidden_inf = answerEncoder(embedded_inp, answer_encoder_hidden_inf)
    
    
    
    if use_cuda:
        answer_outputs = answer_outputs.cuda()
        answer_tags = answer_tags.cuda()


    question_encoder_input = Variable(torch.from_numpy(batch_input[3][batch_number][example_number:example_number+1]))
    if use_cuda:
        question_encoder_input = question_encoder_input.cuda()   #CHECK IF THIS IS RIGHT

    question_encoder_hidden_inf = repackage_hidden(question_encoder_hidden) ##Get this from saved model

    t_answer_mask = Variable(torch.from_numpy(batch_input[3][batch_number][example_number:example_number+1])).float()
    if use_cuda:
        t_answer_mask = t_answer_mask.cuda()

    question_encoder_input = torch.matmul(t_answer_mask[:,0:batch_input[4][batch_num][example_number],:], answer_outputs.float())
    
    _ , question_encoder_hidden_inf = questionEncoder(question_encoder_input[:,0:batch_input[4][batch_number][example_number],:], question_encoder_hidden_inf)

    if type(question_decoder_hidden_inf) == Variable:
        question_decoder_hidden_inf = repackage_hidden(question_decoder_hidden_inf)
    if(type(attention_hidden) == Variable):
        attention_hidden_inf = repackage_hidden(attention_hidden_inf)

    question_loss = 0
    question_decoder_hidden_inf = question_encoder_hidden_inf

    ###embedded_inputs = embedder(torch.from_numpy(batch_input[5][batch_num][i]).long()).cuda()

    maxGenQuesLen = 20
    currGenQuestionLen = 0

    current_question_token = START_TOKEN
    questionGenerated = []
    while currGenQuestionLen < maxGenQuesLen and current_question_token != END_TOKEN:
        question_token_embedding = embedder(Variable(torch.from_numpy(np.array([current_question_token])).long()))
        if use_cuda:
            question_token_embedding = question_token_embedding.cuda()

        if use_attention:
            attn_output, Attention_Weights = attention(question_decoder_hidden_inf.squeeze(0).squeeze(0), attention_hidden_inf.squeeze(0), answer_outputs.squeeze(0))
            decoder_output, attention_hidden_inf = questionDecoder(
                question_token_embedding.unsqueeze(1), attn_output)
        else:
            decoder_output, question_decoder_hidden_inf = questionDecoder(
                question_token_embedding.unsqueeze(1),
                question_decoder_hidden_inf)

        final_output = questionGenerator(decoder_output)
        current_question_token = np.argmax(final_output.data)
        questionGenerated.append(look_up_token_reduced(current_question_token))
        currGenQuestionLen += 1
        
    return questionGenerated

In [ ]:
use_attention = True
use_ground_truth = True
maxDocLenForBatch = max_document_len

answer_encoder_hidden_inf = answerEncoder2.hiddenState
question_encoder_hidden_inf = questionEncoder2.hiddenState
question_decoder_hidden_inf = questionDecoder2.hiddenState
attention_hidden_inf = attention.hidden_state
question_decoder_hidden_inf = questionDecoder2.hiddenState

In [ ]:
use_cuda = True
inp = Variable(torch.from_numpy(batch_input[0][batch_number][example_number:example_number+1]).long())
labels = Variable(torch.from_numpy(batch_input[2][batch_number][example_number:example_number+1])).long() #Let's see if we want to use or not
if use_cuda:
    labels = labels.cuda()

embedded_inp = embedder(inp)

if use_cuda:
    embedded_inp = embedded_inp.cuda()


answer_encoder_hidden_inf = repackage_hidden(answer_encoder_hidden_inf)
answer_tags, answer_outputs, answer_encoder_hidden_inf = answerEncoder(embedded_inp, answer_encoder_hidden_inf)

In [ ]:
if use_cuda:
    answer_outputs = answer_outputs.cuda()
    answer_tags = answer_tags.cuda()


question_encoder_input = Variable(torch.from_numpy(batch_input[3][batch_number][example_number:example_number+1]))
if use_cuda:
    question_encoder_input = question_encoder_input.cuda()   #CHECK IF THIS IS RIGHT

question_encoder_hidden_inf = repackage_hidden(question_encoder_hidden) ##Get this from saved model

t_answer_mask = Variable(torch.from_numpy(batch_input[3][batch_number][example_number:example_number+1])).float()
if use_cuda:
    t_answer_mask = t_answer_mask.cuda()

question_encoder_input = torch.matmul(t_answer_mask[:,0:batch_input[4][batch_num][example_number],:], answer_outputs.float())


In [ ]:
_ , question_encoder_hidden_inf = questionEncoder(question_encoder_input[:,0:batch_input[4][batch_number][example_number],:], question_encoder_hidden_inf)

In [ ]:
if type(question_decoder_hidden_inf) == Variable:
        question_decoder_hidden_inf = repackage_hidden(question_decoder_hidden_inf)
if(type(attention_hidden) == Variable):
    attention_hidden_inf = repackage_hidden(attention_hidden_inf)

question_loss = 0
question_decoder_hidden_inf = question_encoder_hidden_inf

###embedded_inputs = embedder(torch.from_numpy(batch_input[5][batch_num][i]).long()).cuda()

maxGenQuesLen = 20
currGenQuestionLen = 0

current_question_token = look_up_word_reduced("what")
questionGenerated = []

In [ ]:
question_token_embedding = embedder(Variable(torch.from_numpy(np.array([current_question_token])).long()))
if use_cuda:
    question_token_embedding = question_token_embedding.cuda()

if use_attention:
    attn_output, Attention_Weights = attention(question_decoder_hidden_inf.squeeze(0).squeeze(0), attention_hidden_inf.squeeze(0), answer_outputs.squeeze(0))
    decoder_output, attention_hidden_inf = questionDecoder(
        question_token_embedding.unsqueeze(1), attn_output)
else:
    decoder_output, question_decoder_hidden_inf = questionDecoder(
        question_token_embedding.unsqueeze(1),
        question_decoder_hidden_inf)

final_output = questionGenerator(decoder_output)
current_question_token = np.argmax(final_output.data)
questionGenerated.append(look_up_token_reduced(current_question_token))
currGenQuestionLen += 1

In [ ]:
look_up_token_reduced(np.argsort(final_output.data[0][0])[-4])

In [ ]:
printQuestion(batch_number, example_number)

In [ ]:
input = question_decoder_hidden_inf.squeeze(0).squeeze(0)
hidden = attention_hidden_inf.squeeze(0)
encoder_outputs = answer_outputs.squeeze(0)
hidden = hidden.squeeze(0)
hidden = attention2.dropout(hidden)


In [ ]:
.shape

In [ ]:
concat = torch.cat((input.unsqueeze(0), hidden.unsqueeze(0)), 1)
concat_reduced = attention.attn_reduce(concat)
attn_weights = F.softmax(concat_reduced, dim=1)


In [ ]:
attn_weights.squeeze(0).unsqueeze(1).shape, encoder_outputs.shape

In [ ]:
attn_applied = torch.mm(encoder_outputs, attn_weights.squeeze(0).unsqueeze(1))


In [ ]:

##########ONly Batch

verboseBatchPrinting = True
averageBatchLossPrinting = True

num_epochs = 100
answer_encoder_hidden = answerEncoder.initHidden()
question_encoder_hidden = questionEncoder.initHidden()
question_decoder_hidden = None
for epoch in range(1, num_epochs+1):
    avg_loss = 0
    for batch_num in range(len(batch_input[0])):
        
        current_batch_size = len(batch_input[0][batch_num])
        if current_batch_size != batch_size:
            continue
        
        
        maxDocLenForBatch = int(max(batch_input[1][batch_num]))
        mask = np.zeros((current_batch_size, maxDocLenForBatch))
        for i in range(current_batch_size):
            mask[i][0:batch_input[1][0][i]] = 1

            
        inp = Variable(torch.from_numpy(batch_input[0][batch_num]).long())

        labels = Variable(torch.from_numpy(batch_input[2][batch_num])).long()
        if use_cuda:
            labels = labels.cuda()

        ################################### Answer Encoder + Tagging    

        optimizer.zero_grad()
        embedded_inp = embedder(inp)
        if use_cuda:
            embedded_inp = embedder(inp).cuda()

            
        
        answer_encoder_hidden = repackage_hidden(answer_encoder_hidden)
        answer_tags, answer_outputs, answer_encoder_hidden = answerEncoder(embedded_inp, answer_encoder_hidden)
        

        
        
        if use_cuda:
            answer_outputs = answer_outputs.cuda()
            answer_tags = answer_tags.cuda()

        #answer_tags.requires_grad=False
        #answer_outputs.requires_grad=False
        
        
        t_document_mask = Variable(torch.from_numpy(mask)).float()
        if use_cuda:
            t_document_mask = t_document_mask.cuda()
        #t_document_mask.requires_grade = False
        
        outputs = answer_tags.squeeze(-1) * t_document_mask
        #outputs.requires_grad = False
        
        
        answer_loss = criterion1(outputs, labels.float())
        ################################### Q Encoder    
        
        t_answer_mask = Variable(torch.from_numpy(batch_input[3][batch_num])).float()
        if use_cuda:
            t_answer_mask = t_answer_mask.cuda()

        #t_answer_mask.requires_gradui = False
        
        # masking the non-answer embeddings
        question_encoder_input = torch.matmul(t_answer_mask, answer_outputs.float())
        #question_encoder_input.requires_grad = False
        question_encoder_output = Variable(torch.zeros(1,batch_size,questionEncoder.hidden_size))
        #question_encoder_output.requires_grad = False
        
        
        if use_cuda:
            question_encoder_hidden = question_encoder_hidden_batch.cuda()
            
        question_encoder_hidden = repackage_hidden(question_encoder_hidden)
        question_encoder_output, question_encoder_hidden = questionEncoder(question_encoder_input, question_encoder_hidden)
        
        #question_encoder_output.requires_grad = False

        
#         question_encoder_output = Variable(torch.zeros(batch_size, max_question_len, questionEncoder.hidden_size))
#         question_encoder_hidden = Variable(torch.zeros(batch_size, max_question_len, questionEncoder.hidden_size))
        
#         for i in range(current_batch_size):
#             question_encoder_output, question_encoder_hidden = questionEncoder(question_encoder_input[i:i+1,0:batch_input[4][batch_num][i],:], question_encoder_hidden)
#             question_encoder_hidden_batch[:,i:i+1,:] = question_encoder_hidden    

        ################################### Q Decoder    
        
        question_loss = 0
        
        question_decoder_hidden = question_encoder_hidden
        question_decoder_hidden = repackage_hidden(question_decoder_hidden)
        
        embedded_inputs = embedder(torch.from_numpy(batch_input[5][batch_num]).long())
        #embedded_inputs.requires_grad = False
        
        output_labels = Variable(torch.from_numpy(batch_input[6][batch_num]).long())
        
        decoder_output = Variable(torch.zeros(1,batch_size,questionDecoder.hidden_size))
        
        if use_cuda:
            embedded_inputs = embedded_inputs.cuda()
            output_labels = output_labels.cuda()
            
        
        question_decoder_hidden = repackage_hidden(question_decoder_hidden)
        for quesL in range(max(batch_input[7][batch_num])):
            decoder_output, question_decoder_hidden = questionDecoder(
                embedded_inputs[:,quesL:quesL+1,:],
                question_decoder_hidden)

            final_output = questionGenerator(decoder_output)

            output_label = Variable(torch.zeros(32,2000))
            if use_cuda:
                output_label = output_label.cuda()
            for b in range(len(batch_input[5][0])):
                output_label[b,batch_input[5][batch_num][b][quesL]] = 1
            #question_loss += criterion2(final_output.squeeze(1), 
            #                           output_labels[:,quesL:quesL+1])
            question_loss += criterion2(final_output.squeeze(0), output_label)
        
    
        
    
        net_loss = answer_loss + question_loss
        net_loss.backward()
    
        optimizer.step()
        
        avg_loss+= net_loss.data[0]
        if verboseBatchPrinting:
            print ('Batch: %d \t Epoch : %d\tNet Loss: %.4f \tAnswer Loss: %.4f \tQuestion Loss: %.4f' 
                   %(batch_num, epoch, net_loss.data[0], answer_loss.data[0], question_loss.data[0]))
            
    torch.cuda.empty_cache()
          
        
    if averageBatchLossPrinting:
        print('Average Loss after Epoch %d : %.4f'
                   %(epoch, avg_loss/number_of_batches))